<a href="https://www.kaggle.com/soumyacs/sentiment-analysis-naive-bayes?scriptVersionId=84814787" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
import numpy as np
import pandas as pd

In [2]:
# Only need the tweets and associated sentiment
tweets_df = pd.read_csv('../input/covid-19-nlp-text-classification/Corona_NLP_train.csv', encoding='latin-1')
tweets_df = tweets_df[['OriginalTweet', 'Sentiment']]

test_df = pd.read_csv('../input/covid-19-nlp-text-classification/Corona_NLP_test.csv')
test_df = test_df[['OriginalTweet', 'Sentiment']]

In [3]:
test_df

,OriginalTweet,Sentiment
0,TRENDING: New Yorkers encounter empty supermar...,Extremely Negative
1,When I couldn't find hand sanitizer at Fred Me...,Positive
2,Find out how you can protect yourself and love...,Extremely Positive
3,#Panic buying hits #NewYork City as anxious sh...,Negative
4,#toiletpaper #dunnypaper #coronavirus #coronav...,Neutral
...,...,...
3793,Meanwhile In A Supermarket in Israel -- People...,Positive
3794,Did you panic buy a lot of non-perishable item...,Negative
3795,Asst Prof of Economics @cconces was on @NBCPhi...,Neutral
3796,Gov need to do somethings instead of biar je r...,Extremely Negative


In [4]:
# Check for null values
for column in tweets_df.columns:
    print(tweets_df[column].isnull().value_counts())
print()  
for column in test_df.columns:
    print(test_df[column].isnull().value_counts())

False    41157
Name: OriginalTweet, dtype: int64
False    41157
Name: Sentiment, dtype: int64

False    3798
Name: OriginalTweet, dtype: int64
False    3798
Name: Sentiment, dtype: int64


In [5]:
# Check for duplicate values
print(tweets_df.duplicated().value_counts())
print()
print(test_df.duplicated().value_counts())

False    41157
dtype: int64

False    3798
dtype: int64


# Pre-processing

In [6]:
import nltk
from nltk.corpus import stopwords
#from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
import string

nltk.download('stopwords')
#ps = PorterStemmer()
lemmatizer = WordNetLemmatizer()

[nltk_data] Downloading package stopwords to /usr/share/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [7]:
!pip install pyspellchecker 
from spellchecker import SpellChecker
spell = SpellChecker()

     |████████████████████████████████| 2.7 MB 594 kB/s            


In [8]:
'''def textClean(text):
    nopunc = [char.lower() for char in text if char not in string.punctuation]
    nopunc = ''.join(nopunc)
    #nodigit = ''.join([char for char in nopunc if not char.isdigit()])
    tokens = word_tokenize(nopunc)
    nohttp = [word for word in tokens if word[0:4]!='http']
    nostop = [word for word in nohttp if word not in stopwords.words('english')]
    #stemmed = [ps.stem(word) for word in nostop ]
    lemmatized = [lemmatizer.lemmatize(word) for word in nostop]
    return lemmatized'''

"def textClean(text):\n    nopunc = [char.lower() for char in text if char not in string.punctuation]\n    nopunc = ''.join(nopunc)\n    #nodigit = ''.join([char for char in nopunc if not char.isdigit()])\n    tokens = word_tokenize(nopunc)\n    nohttp = [word for word in tokens if word[0:4]!='http']\n    nostop = [word for word in nohttp if word not in stopwords.words('english')]\n    #stemmed = [ps.stem(word) for word in nostop ]\n    lemmatized = [lemmatizer.lemmatize(word) for word in nostop]\n    return lemmatized"

In [9]:
puncs_ = string.punctuation.replace('@','')
puncs = puncs_.replace('#','')
puncs

'!"$%&\'()*+,-./:;<=>?[\\]^_`{|}~'

In [10]:
s = '           @ the springs theatre httpstcoaertookvav'
mytext = " ".join(s.split("  "))
mytext

'      @ the springs theatre httpstcoaertookvav'

In [11]:
def textClean(text):
    # convert to lowercase
    lower = [char.lower() for char in text if char not in puncs]
    lower = ''.join(lower)
    lower = ' '.join(lower.split())
    
    # delete @mentions and #tags
    for char in lower:
        if lower.find('@')==-1 and lower.find('#')==-1: # break loop once @ and # is over
            break
        if (char=='@' or char=='#'):
            try:
                char_index = lower.index(char)
            except ValueError:
                #print(lower)
                break
            del_word = ''
            while char not in string.whitespace:
                del_word = del_word+lower[char_index]
                char_index = char_index + 1
                try:
                    char = lower[char_index] #trying for indexerror incase it is the last character of string
                except IndexError:
                    char = ' '
                except:
                    print("Something else went wrong")
            lower = lower.replace(del_word,'',1)
    lower = [char for char in lower if char not in string.punctuation and char not in string.digits]
    lower = ''.join(lower)
    
    tokens = word_tokenize(lower)
    nohttp = [word for word in tokens if word[0:4]!='http']
    nostop = [word for word in nohttp if word not in stopwords.words('english')]
    return nostop

In [12]:
#print(tweets_df.OriginalTweet[0:10])
#tweets_df.OriginalTweet[0:10].apply(textClean)
temp_list = tweets_df.OriginalTweet[0:10].apply(textClean)
for each_list in temp_list:
    print(each_list)#for word in each_list:

[]
['advice', 'talk', 'neighbours', 'family', 'exchange', 'phone', 'numbers', 'create', 'contact', 'list', 'phone', 'numbers', 'neighbours', 'schools', 'employer', 'chemist', 'gp', 'set', 'online', 'shopping', 'accounts', 'poss', 'adequate', 'supplies', 'regular', 'meds', 'order']
['coronavirus', 'australia', 'woolworths', 'give', 'elderly', 'disabled', 'dedicated', 'shopping', 'hours', 'amid', 'covid', 'outbreak']
['food', 'stock', 'one', 'empty', 'please', 'dont', 'panic', 'enough', 'food', 'everyone', 'take', 'need', 'stay', 'calm', 'stay', 'safe']
['ready', 'go', 'supermarket', 'outbreak', 'im', 'paranoid', 'food', 'stock', 'litteraly', 'empty', 'serious', 'thing', 'please', 'dont', 'panic', 'causes', 'shortage']
['news', 'regionâ\x92s', 'first', 'confirmed', 'covid', 'case', 'came', 'sullivan', 'county', 'last', 'week', 'people', 'flocked', 'area', 'stores', 'purchase', 'cleaning', 'supplies', 'hand', 'sanitizer', 'food', 'toilet', 'paper', 'goods', 'reports']
['cashier', 'grocery

# Model Training

In [13]:
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer(analyzer=textClean)
message = vectorizer.fit_transform(tweets_df['OriginalTweet'])
message.shape

(41157, 39097)

In [14]:
#split the data into 80% training and 20% testing
from sklearn.model_selection import train_test_split
xtrain, xtest, ytrain, ytest = train_test_split(message,tweets_df.Sentiment,test_size=0.20,random_state=0)

In [15]:
# create and train the Naive Bayes Classifier
from sklearn.naive_bayes import MultinomialNB
classifier = MultinomialNB().fit(xtrain, ytrain)

In [16]:
print(classifier.predict(xtest))
print(ytest.values)

['Positive' 'Positive' 'Positive' ... 'Positive' 'Neutral' 'Positive']
['Neutral' 'Negative' 'Positive' ... 'Neutral' 'Neutral' 'Positive']


In [17]:
# Evaluating the model on the training data set
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
pred = classifier.predict(xtrain)
print(classification_report(ytrain, pred))
print()
print("Confusion Matrix: \n", confusion_matrix(ytrain, pred))
print("Accuracy: \n", accuracy_score(ytrain, pred))

                    precision    recall  f1-score   support

Extremely Negative       0.88      0.66      0.76      4387
Extremely Positive       0.83      0.72      0.77      5293
          Negative       0.70      0.78      0.74      7931
           Neutral       0.93      0.55      0.69      6187
          Positive       0.63      0.87      0.73      9127

          accuracy                           0.73     32925
         macro avg       0.79      0.71      0.74     32925
      weighted avg       0.77      0.73      0.73     32925


Confusion Matrix: 
 [[2909   32 1033   36  377]
 [  21 3791  166   20 1295]
 [ 257  152 6180  110 1232]
 [  59  139  887 3372 1730]
 [  64  477  575   99 7912]]
Accuracy: 
 0.7339104024297646


In [18]:
# Evaluating the model on the testing data set
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
pred = classifier.predict(xtest)
print(classification_report(ytest, pred))
print()
print("Confusion Matrix: \n", confusion_matrix(ytest, pred))
print("Accuracy: \n", accuracy_score(ytest, pred))

                    precision    recall  f1-score   support

Extremely Negative       0.59      0.39      0.47      1094
Extremely Positive       0.56      0.44      0.49      1331
          Negative       0.44      0.50      0.47      1986
           Neutral       0.67      0.34      0.45      1526
          Positive       0.41      0.61      0.49      2295

          accuracy                           0.48      8232
         macro avg       0.53      0.46      0.47      8232
      weighted avg       0.51      0.48      0.47      8232


Confusion Matrix: 
 [[ 422    8  511   26  127]
 [   7  589   78   23  634]
 [ 209   72 1000   81  624]
 [  30   44  313  518  621]
 [  48  336  395  123 1393]]
Accuracy: 
 0.47643343051506315


# Test Data

In [19]:
test_df.shape

(3798, 2)

In [20]:
message2 = vectorizer.transform(test_df['OriginalTweet'])
message2.shape

(3798, 39097)

In [21]:
message2

<3798x39097 sparse matrix of type '<class 'numpy.int64'>'
	with 58205 stored elements in Compressed Sparse Row format>

In [22]:
print(classifier.predict(message2))
print(test_df.Sentiment)

['Negative' 'Positive' 'Extremely Positive' ... 'Neutral'
 'Extremely Negative' 'Positive']
0       Extremely Negative
1                 Positive
2       Extremely Positive
3                 Negative
4                  Neutral
               ...        
3793              Positive
3794              Negative
3795               Neutral
3796    Extremely Negative
3797    Extremely Positive
Name: Sentiment, Length: 3798, dtype: object


In [23]:
pred = classifier.predict(message2)
print(classification_report(test_df.Sentiment, pred))
print()
print("Confusion Matrix: \n", confusion_matrix(test_df.Sentiment, pred))
print("Accuracy: \n", accuracy_score(test_df.Sentiment, pred))

                    precision    recall  f1-score   support

Extremely Negative       0.59      0.30      0.39       592
Extremely Positive       0.63      0.35      0.45       599
          Negative       0.44      0.51      0.47      1041
           Neutral       0.66      0.21      0.31       619
          Positive       0.37      0.69      0.48       947

          accuracy                           0.45      3798
         macro avg       0.54      0.41      0.42      3798
      weighted avg       0.51      0.45      0.43      3798


Confusion Matrix: 
 [[175   5 328   4  80]
 [  5 211  34   2 347]
 [ 83  21 529  38 370]
 [  9  15 158 127 310]
 [ 26  84 164  21 652]]
Accuracy: 
 0.4460242232754081
